In [51]:
import numpy as np
matrix = np.array([
    [6.20, 1.10, 0.94, 1.21],
    [1.10, 4.10, 1.30, 0.16],
    [0.94, 1.30, 7.40, 1.10],
    [1.21, 0.16, 1.10, 9.10]
])

In [52]:
matrix = np.array([
    [2.2, 1, 0.5, 2],
    [1, 1.3, 2, 1],
    [0.5, 2, 0.5, 1.6],
    [2, 1, 1.6, 2]
])

In [55]:
def Danilevsky(matrix):
    n = len(matrix)
    Ps = np.zeros((n, n, n))
    Ps[-1] = matrix.copy()
    Ms = np.array(list(np.identity(n)for i in range(n-1)))
    Ms_1 = np.array(list(np.identity(n)for i in range(n-1)))
    M = None
    for k in range(n-2, -1, -1):
    # print(Ps[k+1][k+1])
    # print(Ps[k+1][k+1][k])
        Ms[k][k] = -Ps[k+1][k+1] / Ps[k+1][k+1][k]
        Ms[k][k][k] = -Ms[k][k][k] / Ps[k+1][k+1][k]

        Ms_1[k][k] = Ps[k+1][k+1]

        Ps[k] = Ms_1[k] @ Ps[k+1] @ Ms[k]
        if M is None:
            M = Ms[k]
        else:
            M @= Ms[k]

    # print(Ms[k])
    # print(Ms_1[k])
    # print(Ps[k])

    return Ps[0][0], M

Danilevsky(matrix)

(array([  6.    ,   0.2   , -12.735 ,   2.7616]),
 array([[-0.23112481,  1.07858243,  1.65100154, -1.1587057 ],
        [ 0.08124387, -0.13671383, -1.64095812, -0.27390951],
        [ 0.23812859, -1.2627819 , -0.4131531 ,  0.36957557],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]))